In [1]:
import os
import re
import PyPDF2
import nltk
from sentence_transformers import SentenceTransformer, util

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

c:\Users\danibacaicoa\Desktop\VS projects\Library_manager-1\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\danibacaicoa\Desktop\VS projects\Library_manager-1\.venv\Lib\site-packages\ipykerne

True

In [2]:
folder = 'H:\Mi unidad\Biblioteca\Mathematics\Analysis'  
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\danibacaicoa\AppData\Local\Temp\ipykernel_16520\201937189.py:1: SyntaxWarning: invalid escape sequence '\M'
  folder = 'H:\Mi unidad\Biblioteca\Mathematics\Analysis'
c:\Users\danibacaicoa\Desktop\VS projects\Library_manager-1\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danibacaicoa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develope

In [3]:
folder

'H:\\Mi unidad\\Biblioteca\\Mathematics\\Analysis'

In [14]:
def extract_text_from_pdfs(pdf_folder):
    pdf_texts = {}
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ''
                for page_num in range(len(reader.pages)):
                    text += reader.pages[page_num].extract_text()
                    
                pdf_texts[filename] = text
    return pdf_texts

In [16]:
Text = extract_text_from_pdfs(folder)